In [1]:
import pandas as pd
import numpy as np
import random
import os
import sys
sys.path.append('../')
from modules import utils, constants
import warnings
warnings.filterwarnings('ignore')
import tensorflow
from stable_baselines.common.callbacks import CheckpointCallback

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tensorflow.set_random_seed(constants.SEED)

#### The data

In [3]:
# train_df = pd.read_csv('../data/25_jan/train_set_basic.csv')
train_df = pd.read_csv('../data/missingness/0/training_set.csv')
train_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,1,0,0,0,0,0,0,1,1,0,...,0,1,0,0,0,0,0,1,1,1
4,1,0,0,1,1,1,0,0,0,0,...,0,1,0,0,0,1,0,0,1,1


In [4]:
train_df.cutaneous_lupus.value_counts()

0    44200
3     3120
1     2246
2      834
Name: cutaneous_lupus, dtype: int64

In [5]:
train_df[(train_df.ana==0) & (train_df.label==1)]

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label


In [6]:
train_df.isna().sum()

ana                             0
fever                           0
leukopenia                      0
thrombocytopenia                0
auto_immune_hemolysis           0
delirium                        0
psychosis                       0
seizure                         0
non_scarring_alopecia           0
oral_ulcers                     0
cutaneous_lupus                 0
pleural_effusion                0
pericardial_effusion            0
acute_pericarditis              0
joint_involvement               0
proteinuria                     0
anti_cardioliphin_antibodies    0
anti_β2gp1_antibodies           0
lupus_anti_coagulant            0
low_c3                          0
low_c4                          0
anti_dsdna_antibody             0
anti_smith_antibody             0
label                           0
dtype: int64

In [7]:
len(train_df)

50400

In [8]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_train, y_train = np.array(X_train), np.array(y_train)
X_train.shape, y_train.shape

((50400, 23), (50400,))

In [9]:
action_list = list(constants.CLASS_DICT.keys()) + [col  for col in train_df.columns if col!='label']
action_list

['No lupus',
 'Lupus',
 'Inconclusive diagnosis',
 'ana',
 'fever',
 'leukopenia',
 'thrombocytopenia',
 'auto_immune_hemolysis',
 'delirium',
 'psychosis',
 'seizure',
 'non_scarring_alopecia',
 'oral_ulcers',
 'cutaneous_lupus',
 'pleural_effusion',
 'pericardial_effusion',
 'acute_pericarditis',
 'joint_involvement',
 'proteinuria',
 'anti_cardioliphin_antibodies',
 'anti_β2gp1_antibodies',
 'lupus_anti_coagulant',
 'low_c3',
 'low_c4',
 'anti_dsdna_antibody',
 'anti_smith_antibody']

#### The Model

In [11]:
# training_env = LupusEnv(X_train, y_train)
# training_env = DummyVecEnv([lambda: training_env])
# training_env = VecNormalize(training_env, norm_obs=True, norm_reward=False, clip_obs=10.)

model = utils.create_stable_acktr(X_train, y_train)
checkpoint_callback = CheckpointCallback(save_freq=100000, save_path='../models/logs/acktr', 
                                         name_prefix='acktr2_basic_')

model.learn(total_timesteps=300000000, log_interval=100000, callback=checkpoint_callback)

# Save the trained DQN agent
model.save('../models/22_mar/acktr2_lupus_diagnosis')
# training_env.close()

Wrapping the env in a DummyVecEnv.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
---------------------------------
| ep_len_mean        | 2.83     |
| ep_reward_mean     | 0.606    |
| explained_variance | -0.0386  |
| fps                | 70       |
| nupdates           | 1        |
| policy_entropy     | 3.26     |
| policy_loss        | 2.71     |
| total_timesteps    | 20       |
| value_loss         | 1.46     |
---------------------------------
---------------------------------
| ep_len_mean        | 3.8      |
| ep_reward_mean     | 0.407    |
| explained_variance | -0.0048  |
| fps                | 3117     |
| nupdates           | 100000   |
| policy_entropy     | 1.23     |
| policy_loss        | -0.892   |
| total_timesteps    | 2000000  |
| value_loss         | 1.2

----------------------------------
| ep_len_mean        | 3.79      |
| ep_reward_mean     | 0.487     |
| explained_variance | -0.000296 |
| fps                | 2592      |
| nupdates           | 1900000   |
| policy_entropy     | 1.3       |
| policy_loss        | -1.64     |
| total_timesteps    | 38000000  |
| value_loss         | 2.19      |
----------------------------------
---------------------------------
| ep_len_mean        | 3.98     |
| ep_reward_mean     | 0.421    |
| explained_variance | 0.457    |
| fps                | 2627     |
| nupdates           | 2000000  |
| policy_entropy     | 1.22     |
| policy_loss        | 0.0198   |
| total_timesteps    | 40000000 |
| value_loss         | 0.00404  |
---------------------------------
---------------------------------
| ep_len_mean        | 4.47     |
| ep_reward_mean     | 0.444    |
| explained_variance | 0.0101   |
| fps                | 2658     |
| nupdates           | 2100000  |
| policy_entropy     | 1.07     |
| p

----------------------------------
| ep_len_mean        | 2.65      |
| ep_reward_mean     | 0.645     |
| explained_variance | -0.0767   |
| fps                | 2976      |
| nupdates           | 4100000   |
| policy_entropy     | 1.41e-08  |
| policy_loss        | -1.41e-10 |
| total_timesteps    | 82000000  |
| value_loss         | 0.668     |
----------------------------------
----------------------------------
| ep_len_mean        | 2.82      |
| ep_reward_mean     | 0.679     |
| explained_variance | 0.022     |
| fps                | 2991      |
| nupdates           | 4200000   |
| policy_entropy     | 1.42e-08  |
| policy_loss        | -1.42e-10 |
| total_timesteps    | 84000000  |
| value_loss         | 0.496     |
----------------------------------
----------------------------------
| ep_len_mean        | 2.83      |
| ep_reward_mean     | 0.519     |
| explained_variance | -0.00508  |
| fps                | 3005      |
| nupdates           | 4300000   |
| policy_entropy    

----------------------------------
| ep_len_mean        | 2.76      |
| ep_reward_mean     | 0.301     |
| explained_variance | 0.000653  |
| fps                | 3197      |
| nupdates           | 6300000   |
| policy_entropy     | 2.3e-07   |
| policy_loss        | -2.3e-09  |
| total_timesteps    | 126000000 |
| value_loss         | 1.08      |
----------------------------------
----------------------------------
| ep_len_mean        | 3.67      |
| ep_reward_mean     | 0.531     |
| explained_variance | 0.0981    |
| fps                | 3200      |
| nupdates           | 6400000   |
| policy_entropy     | 0.31      |
| policy_loss        | -0.123    |
| total_timesteps    | 128000000 |
| value_loss         | 0.44      |
----------------------------------
----------------------------------
| ep_len_mean        | 3.65      |
| ep_reward_mean     | 0.492     |
| explained_variance | -0.00693  |
| fps                | 3203      |
| nupdates           | 6500000   |
| policy_entropy    

InvalidArgumentError: Self-adjoint eigen decomposition was not successful. The input might not be valid.
	 [[node kfac_apply/cond/SelfAdjointEigV2_6 (defined at /home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/stable_baselines/acktr/kfac.py:621) ]]

Caused by op 'kfac_apply/cond/SelfAdjointEigV2_6', defined at:
  File "/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/traitlets/config/application.py", line 1043, in launch_instance
    app.start()
  File "/home/lmuyama/anaconda3/envs